In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher as SM
import pandas as pd
import numpy as np
import time
#import Levenshtein
import re
from newsapi import NewsApiClient
import requests
import bs4 as bs
from bs4.element import Comment
from IPython.display import display,HTML
from newsapi import NewsApiClient

In [2]:
#!pip install newsapi-python

In [3]:
data = pd.read_csv('dataXData.csv')
company = data['Business Name'].unique()
company = company.astype(str)
#data.head()

In [4]:
pd.unique(data[['City']].values.ravel())

array([nan, 'quebec', 'bc', ..., 'temperance', 'delmont', 'monaca'],
      dtype=object)

In [5]:
def nameCleaner(name): 
    name = name.lower()
    name = name.split()
    #eng_stopwords = set(stopwords.words("english"))
    #name = [w for w in name if not w in eng_stopwords]
    company_stopwords = set(['group', 'company', 'limited', 'cos', 'dba', 'lllp', 'ltd', 'sa', 'ag', 'pllc', 'pl', 'nv', 'ltcc', 'bv', 'gmblt', 'llc', 'sia', 'sta', 'inc', 'co', 'corp', 'pte'])
    a= []
    for w in name:
        w = re.sub('[^a-zA-Z0-9]+', '',w)
        if(w not in company_stopwords):
            a.append(w)
    name = ''.join(a)
    return name

In [6]:
def text_from_html(body):
    soup = bs.BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    # encode to get rid of encoding errors (hopefully)
    return " ".join(t.strip() for t in visible_texts).encode('utf-8','ignore')

In [7]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [8]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [48]:
start=time.time()
response='y'
while(response=='y'):
    a=[]
    entityInput =input("Which text do you want to search? ")
    stateInput = input("Which country do you want to search (USA/UK/Canada)? If searching for specific state use two letter representation.").upper()
    cleanedEntityInput = nameCleaner(entityInput)
    
    if (stateInput == 'UK'):
        filteredData = data[data['State'] == 'UK']
    elif (stateInput == 'Canada'):
        filteredData = data[data['State'] == 'CANADA']
    elif (stateInput == 'USA'):
        filteredData = data[data['State'] == 'USA']
    else:
        filteredData = data[((data['State'] == stateInput) | (data['State'] == 'USA'))]
    
    company = filteredData['Search Term'].unique()
    company = company.astype(str)
    for i in range(0,len(company),1):
        search=company[i].lower()
        a.append(fuzz.token_sort_ratio(cleanedEntityInput,search))

    #Finding the company names with maximum risk score aka fuzzy ratio
    ratio=np.array(a)
    c=np.where(ratio==ratio.max())
    maximum=max(a)
    top = company[c]
    top_list = set(filteredData.loc[filteredData['Search Term'].isin(top)]['Business Name'])
    print("\nRisk Score for", color.BOLD +entityInput + color.END,"is",color.BOLD + str(maximum) + color.END,"for the brands",top_list)
    
    #Decision Logic for Classification
    if maximum>=90:
        print(color.BOLD,color.RED +"High risk"+ color.END)
    elif (maximum<90 and maximum>=60):
        print(color.BOLD,color.YELLOW +"Medium Risk"+ color.END)
    else:
        print(color.BOLD,color.GREEN +"No Risk"+ color.END)

    # News API part begins here
    df=filteredData.loc[filteredData['Search Term'].isin(top)][['Business Name','Industry','City','State']]
    df1=df.drop_duplicates(keep='first')
    display(df1)
    unique_names = []
    for d in top_list:
        lower = d.lower()
        if lower not in unique_names:
            unique_names.append(lower)

    #print(unique_names)

    entity_column = []
    url_column = []
    snippet_column = []
    # Init
    newsapi = NewsApiClient(api_key='a28acbbe0b2a46428906f353d180c5c3')
    all_articles = []
    for entity in unique_names:
        all_articles=newsapi.get_everything(q=entity,
                                             language='en', # necessary to only maintain english? probably for now
                                             sort_by='relevancy',
                                             page=1)["articles"]

        for article in all_articles:
            url = article["url"]        
            source = requests.get(url)
            if source.status_code != 200:
                continue
            text = text_from_html(source.content).decode('utf-8')

            if (' ' not in entity):
                splitted = text.split()
                lower = [x.lower() for x in splitted]

                try:
                    index = lower.index(entity);

                except ValueError:
                    index = -1
                if(index != -1):
                    if(index < 50):
                        article_snippet = " ".join(splitted[0:index+50])
                    else:
                        article_snippet = " ".join(splitted[index-50:index+50])
                    entity_column.append(entity)
                    url_column.append(url)
                    snippet_column.append(article_snippet)
            else:
                indices = [m.start() for m in re.finditer(entity.lower(), text.lower())]

                if not indices: 
                    continue
                index = indices[0]
                if(index < 100):
                    article_snippet = text[0:index+100]
                else:
                    article_snippet = text[index-100:index+100]
                entity_column.append(entity)
                url_column.append(url)
                snippet_column.append(article_snippet)


    columns = {"Entity" : entity_column, "URL": url_column, "Snippet" : snippet_column}
    df_result = pd.DataFrame.from_dict(columns)
    pd.set_option('max_colwidth', 300)
    def make_clickable(val):
    # target _blank to open new window
        return '<a target="_blank" href="{}">{}</a>'.format(val, val)
    if (df_result.empty):
        print(color.BOLD + "Sorry, No Results found on Internet" + color.END)
    else:
        display(df_result.style.format({'URL': make_clickable})) 
    
    #Recording time taken to search the database and NEWS API
    end=time.time()
    print("\nTime taken:",round(end-start,0),"seconds")
    response=input("\nDo you want to search again? [y/n] ")
print(color.BOLD + "\nTHANK YOU!, for searching with us!" + color.END)

Which text do you want to search? amazon
Which country do you want to search (USA/UK/Canada)? If searching for specific state use two letter representation.CA

Risk Score for amazon is 100 for the brands {'Amazon'}
 High risk


,Business Name,Industry,City,State
246396,Amazon,Multiple,san+francisco,CA


,Entity,URL,Snippet
0,amazon,https://www.engadget.com/2018/11/26/amazon-opens-internal-machine-learning-training/,"replacement for Edge 4h ago View Nest services go down for the third time in three weeks 6h ago View Huawei app uses AI to help deaf children read 7h ago View Pixel 3 update addresses multitasking woes 8h ago View Image credit: Marcos del Mazo via Getty Images Save Amazon opens up its internal machine learning training to everyone More than 30 courses are available for free. Mallory Locklear , @mallorylocklear 11.26.18 in Internet Comments 4259 Shares Share Tweet Share Save Marcos del Mazo via Getty Images Amazon announced today that it's making the machine learning courses it uses"
1,amazon,https://lifehacker.com/how-to-get-tickets-for-an-early-screening-of-aquaman-1830537153,Video Skillet Two Cents Vitals Offspring The Upgrade App Directory How I Work Video Skillet Two Cents Vitals Offspring The Upgrade App Directory How I Work How to Get Tickets For an Early Screening of 'Aquaman' Mike Epstein 11/19/18 12:45pm Filed to: Early Screenings Filed to: Early Screenings Early Screenings Amazon Prime Atom DC Comics Aquaman Movies Comics 8 2 Edit Embiggen Send to Editors Promote Share to Kinja Toggle Conversation tools Go to permalink Image: Warner Brothers Amazon announced this week that it will be holding a set of early screenings of Aquaman for Amazon Prime members six days
2,amazon,https://lifehacker.com/how-to-make-and-receive-skype-calls-using-an-amazon-ech-1830643572,Gizmodo Jalopnik Jezebel Kotaku Lifehacker Splinter The Takeout The Root The Onion Clickhole The Inventory Video Skillet Two Cents Vitals Offspring The Upgrade App Directory How I Work Video Skillet Two Cents Vitals Offspring The Upgrade App Directory How I Work How to Make and Receive Skype Calls Using an Amazon Echo Emily Price 11/25/18 4:27pm Filed to: Amazon Alexa Filed to: Amazon Alexa Amazon Alexa Amazon amazon echo 9 1 Edit Embiggen Send to Editors Promote Share to Kinja Toggle Conversation tools Go to permalink Image: Pexels This week Amazon started rolling out Skype support for Amazon Echo devices.
3,amazon,https://www.engadget.com/2018/11/21/amazon-wants-digital-wallet-brick-and-mortar-stores/,"points to a Chromium replacement for Edge 4h ago View Nest services go down for the third time in three weeks 6h ago View Huawei app uses AI to help deaf children read 7h ago View Pixel 3 update addresses multitasking woes 8h ago View Image credit: ASSOCIATED PRESS Save Amazon reportedly wants its digital wallet in brick-and-mortar stores It’s looking to bring Amazon Pay to gas stations and restaurants. Mallory Locklear , @mallorylocklear 11.21.18 in Business Comments 154 Shares Share Tweet Share Save ASSOCIATED PRESS Amazon wants its Amazon Pay digital wallet to be accepted in brick-and-mortar stores, and"
4,amazon,https://lifehacker.com/determine-if-amazon-product-reviews-are-legit-using-thi-1830622508,"The A.V. Club Deadspin Gizmodo Jalopnik Jezebel Kotaku Lifehacker Splinter The Takeout The Root The Onion Clickhole The Inventory Video Skillet Two Cents Vitals Offspring The Upgrade App Directory How I Work Video Skillet Two Cents Vitals Offspring The Upgrade App Directory How I Work Determine if Amazon Product Reviews Are Legit Using This Site Emily Price 11/23/18 4:00pm Filed to: Shopping Filed to: Shopping Shopping Amazon amazon hacks 42 4 Edit Embiggen Send to Editors Promote Share to Kinja Toggle Conversation tools Go to permalink Image: Pexels Holiday shopping time is upon us, which means we’re"
5,amazon,https://www.theverge.com/2018/11/5/18063738/amazon-free-holiday-shipping-no-minimum-spend,"Science Culture Internet TV Film Games Comics Music All Culture Cars Autonomy Aviation Cars Electric cars Mass Transit Ride-Sharing Rideables All Transportation Reviews Phones Laptops Cameras Tablets Headphones Smartwatches This is my Next How-to Good Deals More from Verge Guidebook Longform Video Podcasts Circuit Breaker Store Newsletters Forums ✕ Tech


Time taken: 23.0 seconds

Do you want to search again? [y/n] y
Which text do you want to search? apple
Which country do you want to search (USA/UK/Canada)? If searching for specific state use two letter representation.usa

Risk Score for apple is 100 for the brands {'Apple Inc.'}
 High risk


,Business Name,Industry,City,State
51,Apple Inc.,Information Technology,NaN,USA
2809,Apple Inc.,Computer Manufacturing,NaN,USA


,Entity,URL,Snippet
0,apple inc.,https://www.reuters.com/article/us-apple-products-idUSKCN1NF00Z,"pple Store in Manhattan, New York, U.S., September 21, 2018. REUTERS/Shannon Stapleton (Reuters) - Apple Inc said on Friday it had found some issues affecting some of its iPhone X and 13-inch MacBook"
1,apple inc.,https://www.fastcompany.com/90272180/supreme-court-hears-arguments-over-apple-app-store-monopoly?partner=feedburner&utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+fastcompany%2Fheadlines+%28Fast+Company%29,"e Court today to defend its 30 percent commission on app sales. The justices will hear arguments in Apple Inc. v. Pepper , over whether the tech giant can be forced to pay damages to iPhone owners who"
2,apple inc.,https://www.reuters.com/article/us-apple-iphone-pricing-analysis-idUSKCN1NJ26G,"hangs over Apple Stephen Nellis 8 Min Read SAN FRANCISCO (Reuters) - A raft of profit warnings from Apple Inc ( AAPL.O ) suppliers this week has fueled investor concerns that iPhone sales, in terms of"
3,apple inc.,https://gizmodo.com/should-the-supreme-court-knock-the-first-brick-out-of-a-1830569176,nock the First Brick Out of Apple's Walled Garden? Rhett Jones 11/23/18 10:00am Filed to: Pepper v. Apple Inc Filed to: Pepper v. Apple Inc Pepper v. Apple Inc supreme court apple 158 4 Edit Embiggen
4,apple inc.,https://gizmodo.com/supreme-court-appears-to-lean-heavily-against-apples-de-1830662533,esent at today’s hearing say the case’s outcome isn’t looking great for Apple. The case of Pepper v Apple Inc has taken a complex path to the highest court in the land (which we’ve previously covered
5,apple inc.,https://www.reuters.com/article/us-apple-india-analysis-idUSKCN1NC0HK,", spends hours on his phone watching shows, surfing and shopping, making him the perfect target for Apple Inc as it strives to raise sales among India’s 1.3 billion consumers. But in a country where t"
6,apple inc.,https://www.yahoo.com/news/apple-supplier-aac-tech-third-quarter-profit-dives-050100482--finance.html,"Sijia Jiang HONG KONG (Reuters) - AAC Technologies Holdings Inc, an acoustic components supplier to Apple Inc, on Thursday reported a drop of 28.7 percent to 973 million yuan ($140.38 million) in quar"
7,apple inc.,https://www.marketwatch.com/story/stocks-close-lower-as-dow-logs-longest-losing-streak-in-3-months-2018-11-14,"th the Dow Jones Industrial Average logging its longest losing streak in three months, as blue-chip Apple Inc. flirted with bear-market territory. See full story. PG&E stock and bonds hammered on con"
8,apple inc.,https://www.entrepreneur.com/article/322848,"ere down on the day, as investors appear to be looking for value over growth. Negative sentiment on Apple Inc., which reported disappointing earnings last week, may also be weighing on the market. Aka"
9,apple inc.,https://www.marketwatch.com/story/trumps-saber-rattling-ahead-of-china-trade-talks-hurts-apple-2018-11-26,"ummit meeting with Chinese President Xi Jinping later this week when he said that previously exempt Apple Inc. products — the iPhone and the Mac — could be subject to tariffs, if the upcoming trade ta"



Time taken: 97.0 seconds

Do you want to search again? [y/n] n

THANK YOU!, for searching with us!
